In [ ]:
pip install python-terrier


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 2.8 MB/s 
     |████████████████████████████████| 1.1 MB 23.3 MB/s 
     |████████████████████████████████| 69 kB 7.0 MB/s 
     |████████████████████████████████| 303 kB 58.8 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 45 kB 1.8 MB/s 
     |████████████████████████████████| 126 kB 52.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2 MB 50.5 MB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 294 kB 39.3 MB/s 
     |████████████████████████████████| 285 kB 67.0 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.1-py3-none-any.whl size=104091 sha256=880f1ca2038467a2028759d5e93ff40f89c253528fb99

In [ ]:
import pyterrier as pt
pt.init()


terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading the indexes for Podcast Small

In [ ]:
# Raw Document Index 
sample_set_we = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/sample_without/data.properties")
raw= pt.IndexFactory.of(sample_set_we)

In [ ]:
#Wikipedia Index 
wiki_instance = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raws/data.properties")
wiki_no_difference= pt.IndexFactory.of(wiki_instance)

In [ ]:
#Wikipedia Index Freq 2 
wiki_instance_2 = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raw_freq_2/data.properties")
wiki_no_difference_2= pt.IndexFactory.of(wiki_instance_2)

In [ ]:
#Wikipedia Index Freq 5
wiki_instance_5 = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raw_freq_5_test/data.properties")
wiki_no_difference_5= pt.IndexFactory.of(wiki_instance_5)

In [ ]:
import pandas as pd
id_template = 'spotify:episode:{}_{}'

def get_ep(a,b):
  return id_template.format(a,b)

Raw Index with DCU Approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(raw, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()

combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.25422411493243313,
 'ndcg': 0.5108938918540078,
 'ndcg_cut_30': 0.3248789779907644,
 'P_10': 0.30000000000000004}

Raw Index with QE Approach 

In [ ]:

qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

20:43:11.584 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.2315903093444428, 'ndcg': 0.4733466463739926, 'ndcg_cut_30': 0.2727471259883246, 'P_10': 0.2625}


Raw Index with normal topics and BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(raw, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= bm25 + dph
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DCU Linear Combination Wikipedia no ents ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

DCU Linear Combination Wikipedia no ents 
{'map': 0.2762875222633958, 'ndcg': 0.47840418194031553, 'ndcg_cut_30': 0.2962310222812588, 'P_10': 0.3125}


Raw Index Wikfied Query topics and BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(raw, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= bm25 + dph
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DCU Linear Combination Wikipedia no ents ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

DCU Linear Combination Wikipedia no ents 
{'map': 0.16596425857159985, 'ndcg': 0.42344151900565763, 'ndcg_cut_30': 0.18456556859686715, 'P_10': 0.15000000000000002}


Raw Index with Wikified Query and QE approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query.xml",format="trecxml")


qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

20:46:53.271 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.12187329391878444, 'ndcg': 0.34011751012390323, 'ndcg_cut_30': 0.12832671688728825, 'P_10': 0.09999999999999999}


Raw Index with normal topics and BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

00:07:57.280 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.2711190535202457, 'ndcg': 0.4662204715132263, 'ndcg_cut_30': 0.26783376749757115, 'P_10': 0.3125}


Raw Index with Entities topics and PL2

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")

pl2= pt.BatchRetrieve(raw, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])



res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]


rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)

print("PL 2 ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))


PL 2 
{'map': 0.24968646497395489, 'ndcg': 0.5125724838201344, 'ndcg_cut_30': 0.3231425969086813, 'P_10': 0.2875000000000001}


Wikified Index + Wikified Query on BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()



combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.21231055007066818,
 'ndcg': 0.4980402819212154,
 'ndcg_cut_30': 0.299939861391186,
 'P_10': 0.31250000000000006}

Wikified Index + Normal topics on BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= bm25 + dph
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DCU Linear Combination Wikipedia no ents ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

DCU Linear Combination Wikipedia no ents 
{'map': 0.3170158718879924, 'ndcg': 0.506810722641573, 'ndcg_cut_30': 0.34036510665973313, 'P_10': 0.4125}


Wikified Index + Entity topics on BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()



combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.30435256260000515,
 'ndcg': 0.5547869441575499,
 'ndcg_cut_30': 0.3861063791648928,
 'P_10': 0.4000000000000001}

Wikified Index + Wikfied query + entities  on BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_ent.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()



combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.22052363593952226,
 'ndcg': 0.5069845630459082,
 'ndcg_cut_30': 0.3045576680805741,
 'P_10': 0.36250000000000004}

Wikified Index + entities on PL2 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")

pl2= pt.BatchRetrieve(wiki_no_difference, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.29033770894880706,
 'ndcg': 0.5483741802262013,
 'ndcg_cut_30': 0.3764770587022482,
 'P_10': 0.4}

Wikified Index frequency 2 + entities  on BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")

pl2= pt.BatchRetrieve(wiki_no_difference_2, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.283772900810541,
 'ndcg': 0.5493827155664024,
 'ndcg_cut_30': 0.381256957105417,
 'P_10': 0.4000000000000001}

Wikified Index + Wikfied query + entities  on PL2 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_ent.xml",format="trecxml")

pl2= pt.BatchRetrieve(wiki_no_difference, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.13643267720979396,
 'ndcg': 0.39887434137357997,
 'ndcg_cut_30': 0.22268413928372086,
 'P_10': 0.18750000000000003}

Wikified Index + Normal topics on PL2

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")

pl2= pt.BatchRetrieve(raw, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.27141166156199226,
 'ndcg': 0.4795647014286212,
 'ndcg_cut_30': 0.2957122000210158,
 'P_10': 0.30000000000000004}

Wikified Index + entities  on QE Approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")


qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

21:28:48.370 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.29790216551232757, 'ndcg': 0.5339255819371314, 'ndcg_cut_30': 0.35707615739861853, 'P_10': 0.3750000000000001}


Wikified Index + original topics  on QE approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/topic01.xml",format="trecxml")


qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

21:30:19.734 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.3221721342469178, 'ndcg': 0.5032836990699376, 'ndcg_cut_30': 0.31425262102035617, 'P_10': 0.375}


Wikified Index + Nouns on QE approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns.xml",format="trecxml")


qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "DPH"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

21:36:24.526 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.30419731080228024, 'ndcg': 0.5620822705417831, 'ndcg_cut_30': 0.39023001959357856, 'P_10': 0.37499999999999994}


Wikified Index + nouns + entities  on QE approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth.xml",format="trecxml")


qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "DPH"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

00:20:25.991 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.3170541127120993, 'ndcg': 0.5700382092656799, 'ndcg_cut_30': 0.40804512430695794, 'P_10': 0.39999999999999997}


Wikified Index + nouns + entities  on BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()

combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.3077112225283314,
 'ndcg': 0.5639010522421791,
 'ndcg_cut_30': 0.4210917687766055,
 'P_10': 0.425}

Wikified Index freq 2 + entities  on BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference_2, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference_2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()

combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.2966157030650845,
 'ndcg': 0.5716481465254507,
 'ndcg_cut_30': 0.4055295130386219,
 'P_10': 0.3875000000000001}

Wikified Index + noun + entities  on BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()

combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.3077112225283314,
 'ndcg': 0.5639010522421791,
 'ndcg_cut_30': 0.4210917687766055,
 'P_10': 0.425}